# Advanced Usage

This notebook covers advanced Plotly customization. All kwargs are passed directly to [Plotly Express](https://plotly.com/python/plotly-express/), and figures can be modified using the full [Plotly Graph Objects API](https://plotly.com/python/graph-objects/).

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr

from xarray_plotly import config, xpx

config.notebook()

In [ ]:
# Load sample data
df_stocks = px.data.stocks().set_index("date")
df_stocks.index = df_stocks.index.astype("datetime64[ns]")

stocks = xr.DataArray(
    df_stocks.values,
    dims=["date", "company"],
    coords={"date": df_stocks.index, "company": df_stocks.columns.tolist()},
    name="price",
)

df_gap = px.data.gapminder()
countries = ["United States", "China", "Germany", "Brazil", "Nigeria"]
df_life = df_gap[df_gap["country"].isin(countries)].pivot(
    index="year", columns="country", values="lifeExp"
)
life_exp = xr.DataArray(
    df_life.values,
    dims=["year", "country"],
    coords={"year": df_life.index, "country": df_life.columns.tolist()},
    name="life_exp",
)

## Passing Kwargs to Plotly Express

All keyword arguments are passed directly to [Plotly Express functions](https://plotly.com/python-api-reference/plotly.express.html):

In [ ]:
# All px.line kwargs work: template, labels, colors, etc.
fig = xpx(stocks).line(
    title="Stock Performance",
    template="plotly_white",
    labels={"price": "Normalized Price", "date": "Date", "company": "Ticker"},
    color_discrete_sequence=px.colors.qualitative.Set2,
)
fig

In [ ]:
# px.imshow kwargs: colorscale, midpoint, aspect
# See: https://plotly.com/python/imshow/
life_change = life_exp - life_exp.isel(year=0)

fig = xpx(life_change).imshow(
    color_continuous_scale="RdBu",
    color_continuous_midpoint=0,
    aspect="auto",
    title="Life Expectancy Change Since 1952",
)
fig

In [ ]:
# px.bar kwargs: barmode, text_auto
# See: https://plotly.com/python/bar-charts/
fig = xpx(life_exp.sel(year=[1952, 1982, 2007])).bar(
    barmode="group",
    text_auto=".1f",
    title="Life Expectancy",
)
fig

## Modifying Figures After Creation

All methods return a Plotly `Figure`. Use the [Figure API](https://plotly.com/python/creating-and-updating-figures/) to customize:

### update_layout

Modify [layout properties](https://plotly.com/python/reference/layout/): legend, margins, fonts, etc.

In [ ]:
fig = xpx(stocks).line()

fig.update_layout(
    title={"text": "Stock Prices", "x": 0.5, "font": {"size": 24}},
    legend={
        "orientation": "h",
        "yanchor": "bottom",
        "y": 1.02,
        "xanchor": "center",
        "x": 0.5,
    },
    margin={"t": 80},
)
fig

### update_traces

Modify [trace properties](https://plotly.com/python/reference/): line width, markers, opacity, etc.

In [ ]:
fig = xpx(stocks).line()
fig.update_traces(line={"width": 3})
fig.update_layout(title="Thicker Lines")
fig

In [ ]:
fig = xpx(stocks).scatter()
fig.update_traces(marker={"size": 12, "opacity": 0.7, "line": {"width": 1, "color": "white"}})
fig.update_layout(title="Custom Markers")
fig

### update_xaxes / update_yaxes

Modify [axis properties](https://plotly.com/python/axes/): range slider, tick format, etc.

In [ ]:
fig = xpx(stocks).line(title="With Range Slider")
fig.update_xaxes(rangeslider_visible=True)
fig.update_yaxes(tickformat=".0%")
fig

### Adding Shapes and Annotations

Add reference lines, [shapes](https://plotly.com/python/shapes/), and [annotations](https://plotly.com/python/text-and-annotations/):

In [ ]:
fig = xpx(stocks).line(title="With Reference Lines and Annotations")

fig.add_hline(y=1.0, line_dash="dash", line_color="gray", annotation_text="Baseline")
fig.add_vline(x="2018-10-01", line_dash="dot", line_color="red")

fig.add_annotation(
    x="2018-10-01",
    y=1.4,
    text="Market correction",
    showarrow=True,
    arrowhead=2,
    ax=40,
    ay=-40,
)
fig

### Adding Traces with Graph Objects

Add custom traces using [Plotly Graph Objects](https://plotly.com/python/graph-objects/):

In [ ]:
fig = xpx(stocks.sel(company="GOOG")).line(title="GOOG with Moving Average")

# Add moving average as a new trace
goog = stocks.sel(company="GOOG")
ma_20 = goog.rolling(date=20, center=True).mean()

fig.add_trace(
    go.Scatter(
        x=ma_20.coords["date"].values,
        y=ma_20.values,
        mode="lines",
        name="20-day MA",
        line={"dash": "dash", "color": "red", "width": 2},
    )
)
fig

## Exporting Figures

See [static image export](https://plotly.com/python/static-image-export/) and [HTML export](https://plotly.com/python/interactive-html-export/).

```python
# Interactive HTML
fig.write_html("plot.html")

# Static images (requires: pip install kaleido)
fig.write_image("plot.png", scale=2)
fig.write_image("plot.svg")
fig.write_image("plot.pdf")
```

## More Resources

- [Plotly Express API](https://plotly.com/python-api-reference/plotly.express.html)
- [Figure Reference](https://plotly.com/python/reference/)
- [Plotly Tutorials](https://plotly.com/python/)